In [304]:
import requests
import os
import sys
import pandas as pd
import numpy as np

# import key sklearn ml algos
# import key sklearn metrics
# import 

from typing import Optional
import glob
from scipy import stats
# Add the parent directory of this notebook to sys.path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)

from project_tools import project_utils, project_class

import datetime
import json
from tqdm.notebook import tqdm
import gc
# import ds_utils
import random
import matplotlib.pyplot as plt
%matplotlib inline

from importlib import reload
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


In [ ]:
# check package - lightgbm, xgboost, bayesian-optimization, 

In [23]:
# major technical items to review:
# 1. groupby-aggregator handy func
# 2. numerai validator class with different algo
# 3. null hypo. feature selection routine - if not using the numerai class
# 4. weight optimisation routine 

In [ ]:
# handy code to develop to prep for interview
# 1. final position to target transformation 
# 2. probability normalization for horses in the same race
# 3. prediction optimisation for metric per race 

In [2]:
# generic plan 
# 1. set up package and load data
# 2. initial analysis
# 2.1 dataframe.info 
# 2.2 key columns format, null count
# 2.3 identify data cleaning target
# 3. target analysis    
# 4. generate measurable targets - if needed 
# 4.1 identify "very import features" - do transformation of those if needed 
# 5. generate machine trainable dataset - for tree and for linear & NN algo.
# 6. generate additional features
# 7. 


# to-do:
# 1. create generate data analysis coding blocks including simple visualizations on notebook
# 2. switch to jupyber lab setting, and write the code there 

In [281]:
horse_result_file  = '../data/race-result-horse.csv'
race_result_file = '../data/race-result-race.csv'

horse_df = pd.read_csv(horse_result_file)
race_df = pd.read_csv(race_result_file)

horse_df['finishing_position'] = horse_df['finishing_position'].fillna('NA')

# data cleaning

In [282]:
clean_position = []
for i,row in horse_df.iterrows():
    row_finishing_position = row['finishing_position']
    if 'DH' in row_finishing_position:
        row_position = int(row_finishing_position[0])
    else:
        if str.isdigit(row_finishing_position):
            row_position = int(row_finishing_position)
        else:
            row_position = 99
    clean_position.append(row_position)
horse_df['clean_position'] = clean_position

In [283]:
def text_to_numeric(text):
    """
    Convert text to numeric (int or float), return np.nan if not possible
    
    Args:
        text: Input text/value to convert
        
    Returns:
        int, float or np.nan: Converted numeric value or np.nan if conversion fails
    """
    # Handle None, empty string, or np.nan input
    if text is None or text == '' or (isinstance(text, float) and np.isnan(text)):
        return np.nan
    
    # If input is already numeric, return as is
    if isinstance(text, (int, float)):
        return text
    
    # Convert to string and strip whitespace
    text = str(text).strip()
    
    try:
        # Try integer first
        value = int(text)
        return value
    except ValueError:
        try:
            # Try float
            value = float(text)
            return value
        except ValueError:
            return np.nan

In [284]:
def convert_length_behind(text):
    """
    Convert length behind text to numerical value
    
    Args:
        text (str): Text representation of length behind
        
    Returns:
        float: Numerical value of length behind
    """
    # Special case mappings
    special_cases = {
        '-': 0,
        'N': 0.3,
        'SH': 0.1,
        'HD': 0.25,
        'NOSE': 0.05,
        '+NOSE': 0.75,
        'ML': 50,
        '+SH': 0.15,
        'TO': 0.1,
        '+1/2': 99,
        '---': 99
    }
    
    # Check if input is in special cases
    if text in special_cases:
        return special_cases[text]
    
    try:
        # Try to convert direct number first
        return float(text)
    except ValueError:
        # Handle compound fractions (e.g., "4-1/4", "5-1/2")
        parts = text.split('-')
        
        if len(parts) == 1:
            # Handle pure fractions (e.g., "1/4", "1/2")
            if '/' in parts[0]:
                num, denom = map(float, parts[0].split('/'))
                return num/denom
        else:
            # Handle whole number with fraction
            whole = float(parts[0])
            if '/' in parts[1]:
                num, denom = map(float, parts[1].split('/'))
                return whole + num/denom
            
        raise ValueError(f"Unable to parse length: {text}")

# Example usage:
# lengths = ['2', '4-1/4', '5-1/2', 'NOSE', '1/2', 'N']
# numeric_lengths = [convert_length_behind(x) for x in lengths]

In [285]:
horse_df['clean_actual_weight'] = horse_df['actual_weight'].apply(lambda x: text_to_numeric(x))
horse_df['clean_declared_horse_weight'] = horse_df['declared_horse_weight'].apply(lambda x: text_to_numeric(x))

In [286]:
horse_df['clean_length_behind_winner'] = horse_df['length_behind_winner'].apply(lambda x: convert_length_behind(x))

In [288]:
race_df['race_course_track'] = race_df['race_course'] + '_' + race_df['track']

In [5]:
# target generation

In [287]:
horse_df['isin_winner'] = (horse_df['clean_position'] == 1).astype(int)
horse_df['isin_top3'] = (horse_df['clean_position'] <= 3).astype(int)


# merge data

In [291]:
horse_race_df = horse_df.merge(right=race_df, on=['race_id'], how='left')
horse_race_df.shape

(30189, 37)

# feature generation

In [6]:
# feature note:
# race_course_track: the combination of race_course and track
# race course/track feature - sectional time statistics
# race cource - race course_track conditional ratio
# 
# horse feature -  ratio between actual weight, declared weight
# horse feature - length behind winner statistics
# horse feature - running position statistics
# horse feature - jockey statistics - avg finishing position (<99)
# horse feature - trainer statistic - avg finishing position (<99)
# horse feature - failed to finish ration - num <99 / num_races
 
# horse race feature - finish time/course length
# horse race feature - 



# ask perplexity for features engineering idea

# race course features

In [292]:
# Pattern classification
def classify_race_pattern(times):
    times = times.astype(float)
    diffs = np.diff(times)
    if all(diffs < 0):
        return 'progressive_speedup'
    elif all(diffs > 0):
        return 'progressive_slowdown'
    elif diffs[0] < 0 and diffs[-1] > 0:
        return 'middle_burst'
    elif diffs[0] > 0 and diffs[-1] < 0:
        return 'strong_finish'
    return 'mixed'

In [293]:
race_course_features = []
for i,row in tqdm(race_df.iterrows(), total=len(race_df)):
    stime_items = np.array(row['sectional_time'].split(' ')).astype(float)    
    item_dict = dict()
    item_dict['race_id'] = row['race_id']
    item_dict['stime_min'] = stime_items.min()
    item_dict['stime_max'] = stime_items.max()
    item_dict['stime_mean'] = stime_items.mean()
    item_dict['stime_std'] = stime_items.std()
    stime_diffs = np.diff(stime_items)
    item_dict['stime_diffs'] = stime_diffs
    item_dict['stime_pace_variance'] = np.var(stime_diffs)
    item_dict['stime_max_pace_change'] = np.abs(stime_diffs).max()
    item_dict['stime_skewness'] = stats.skew(stime_items)
    
    item_dict['stime_first_half_avg'] = np.mean(stime_items[:len(stime_items)//2].astype(float))
    item_dict['stime_second_half_avg'] = np.mean(stime_items[len(stime_items)//2:].astype(float))
    item_dict['stime_speed_bias'] = item_dict['stime_second_half_avg'] - item_dict['stime_first_half_avg']  # Negative means faster finish

    # Percentile-based features
    item_dict['stime_percentiles'] = np.percentile(stime_items.astype(float), [25, 75])
    item_dict['stime_iqr'] = item_dict['stime_percentiles'][1] - item_dict['stime_percentiles'][0]

    # Section comparisons
    item_dict['stime_fastest_section_idx'] = np.argmin(stime_items.astype(float))
    item_dict['stime_slowest_section_idx'] = np.argmax(stime_items.astype(float))


    # Trend analysis
    item_dict['is_getting_faster'] = all(stime_diffs < 0)  # True if consistently speeding up
    item_dict['is_getting_slower'] = all(stime_diffs > 0)  # True if consistently slowing down

    item_dict['race_pattern'] = classify_race_pattern(stime_items)
    race_course_features.append(item_dict)    
# break
# stime_

rc_features = pd.DataFrame.from_dict(race_course_features)



  0%|          | 0/2367 [00:00<?, ?it/s]

In [294]:
rc_features.shape

(2367, 19)

In [295]:
rc_features.sample(5)

,race_id,stime_min,stime_max,stime_mean,stime_std,stime_diffs,stime_pace_variance,stime_max_pace_change,stime_skewness,stime_first_half_avg,stime_second_half_avg,stime_speed_bias,stime_percentiles,stime_iqr,stime_fastest_section_idx,stime_slowest_section_idx,is_getting_faster,is_getting_slower,race_pattern
687,2014-688,22.36,23.80,23.126667,0.591570,"[-1.4400000000000013, 0.8599999999999994]",1.322500,1.44,-0.232731,23.800000,22.79,-1.010000,"[22.79, 23.509999999999998]",0.7200,1,0,False,False,middle_burst
382,2014-383,14.12,26.41,23.073333,4.112792,"[9.070000000000002, 1.9499999999999993, 1.2699999999999996, -1.7100000000000009, 0.17999999999999972]",13.496256,9.07,-1.584875,20.816667,25.33,4.513333,"[23.567500000000003, 25.075]",1.5075,0,3,False,False,mixed
891,2015-113,22.84,23.64,23.200000,0.331461,"[-0.8000000000000007, 0.28000000000000114]",0.291600,0.80,0.347974,23.640000,22.98,-0.660000,"[22.98, 23.380000000000003]",0.4000,1,0,False,False,middle_burst
623,2014-623,13.81,24.44,21.618000,3.951432,"[8.74, 1.259999999999998, 0.6300000000000026, -0.9600000000000009]",13.979119,8.74,-1.412526,18.180000,23.91,5.730000,"[22.55, 23.81]",1.2600,0,3,False,False,strong_finish
1049,2015-273,23.61,28.16,25.222500,1.790731,"[-4.550000000000001, 1.5500000000000007, -1.1999999999999993]",6.221667,4.55,0.833371,25.885000,24.56,-1.325000,"[23.872500000000002, 25.91]",2.0375,1,0,False,False,mixed


In [296]:
rc_features.to_parquet('../feature_data/race_course_features.parquet')

In [298]:
def calculate_track_condition_ratios(df):
    # Get all unique track conditions
    res_df = df[['race_id','race_course_track']].copy()
    track_conditions = df['track_condition'].unique()
    
    # Group by race course track and calculate ratios for each condition
    track_counts = df.groupby('race_course_track').size()
    
    # Calculate ratio for each track condition
    for condition in track_conditions:
        condition_counts = df[df['track_condition'] == condition].groupby('race_course_track').size()
        ratio_col_name = f'{condition}_ratio'
        res_df[ratio_col_name] = df['race_course_track'].map(condition_counts / track_counts)
        
    # Fill NaN values with 0 (for tracks that don't have a particular condition)
    condition_ratio_cols = [f'{c}_ratio' for c in track_conditions]
    res_df[condition_ratio_cols] = res_df[condition_ratio_cols].fillna(0)
    output_cols = ['race_id','race_course_track'] + condition_ratio_cols
    return res_df[output_cols]

# Calculate track condition ratios
trackcond_df = calculate_track_condition_ratios(race_df)

# Display first few rows of the dataframe with new ratio columns
print("\nFirst few rows with track condition ratios:")
print(trackcond_df.shape)







First few rows with track condition ratios:
(2367, 11)


In [299]:
trackcond_df.sample(5)

,race_id,race_course_track,GOOD TO FIRM_ratio,WET FAST_ratio,GOOD_ratio,GOOD TO YIELDING_ratio,WET SLOW_ratio,FAST_ratio,YIELDING_ratio,YIELDING TO SOFT_ratio,SOFT_ratio
1998,2016-439,"Happy Valley_TURF - ""C+3"" COURSE",0.342105,0.0,0.638158,0.019737,0.0,0.0,0.000000,0.0,0.0
2211,2016-651,"Sha Tin_TURF - ""C"" COURSE",0.425620,0.0,0.433884,0.111570,0.0,0.0,0.028926,0.0,0.0
249,2014-250,"Happy Valley_TURF - ""B"" COURSE",0.338983,0.0,0.618644,0.042373,0.0,0.0,0.000000,0.0,0.0
520,2014-520,"Sha Tin_TURF - ""B+2"" COURSE",0.632850,0.0,0.314010,0.038647,0.0,0.0,0.014493,0.0,0.0
350,2014-351,"Happy Valley_TURF - ""C+3"" COURSE",0.342105,0.0,0.638158,0.019737,0.0,0.0,0.000000,0.0,0.0


In [300]:
trackcond_df.to_parquet('../feature_data/trace_condition.parquet')

# horse features

In [ ]:
# horse feature -  ratio between actual weight, declared weight
# horse feature - length behind winner statistics
# horse feature - running position statistics
# horse feature - jockey statistics - avg finishing position (<99)
# horse feature - trainer statistic - avg finishing position (<99)
# horse feature - failed to finish ration - num <99 / num_races

In [301]:
horse_feats = horse_df[['horse_id']].copy()
horse_feats['awght_dwght_ratio'] = horse_df['clean_actual_weight'] / horse_df['clean_declared_horse_weight']
horse_feats['awght_dwght_delta'] = horse_df['clean_actual_weight'] - horse_df['clean_declared_horse_weight']

In [302]:
 def get_groupbystat(data, groupby_feats, numfeats, stat):
    roundmetric_agg_rcp = [
        [groupby_feats, numfeats1, stat],
    ]

    res = project_utils.groupby_agg_execution(roundmetric_agg_rcp, data)['model']
    rename_dict = {}
    for c in res.columns.tolist():
        if c != groupby_feat:
            rename_dict[c] = c[6:] # remove 'model_' in column name
    res.rename(columns = rename_dict, inplace=True)
    return res

In [317]:
reload(project_utils)

<module 'project_tools.project_utils' from '/home/yifan/playground/hk_horse_racing/project_tools/project_utils.py'>

In [318]:
groupby_feats = ['horse_id', 'race_course_track']
numfeats = ['running_position_1', 'running_position_2', 'running_position_3', 'running_position_4']
stat = ['sum', 'mean', 'count', 'std',
         {'consistency': project_utils.get_array_sharpe}]  # {'ptp':np.ptp}]#{'sharp':project_utils.get_array_sharpe}]


agg_recipies = [
    [groupby_feats, numfeats, stat],
]
res_dfs = project_utils.groupby_agg_execution(agg_recipies, horse_race_df, stat)


generating statistic horse_id_race_course_track_running_position_1_sum
generating statistic horse_id_race_course_track_running_position_1_mean
generating statistic horse_id_race_course_track_count
generating statistic horse_id_race_course_track_running_position_1_std
generating statistic horse_id_race_course_track_running_position_1_consistency


/home/yifan/playground/hk_horse_racing/project_tools/project_utils.py:540: RuntimeWarning: divide by zero encountered in scalar divide
  elif operator == 'division':


generating statistic horse_id_race_course_track_running_position_2_sum
generating statistic horse_id_race_course_track_running_position_2_mean
generating statistic horse_id_race_course_track_count
generating statistic horse_id_race_course_track_running_position_2_std
generating statistic horse_id_race_course_track_running_position_2_consistency


/home/yifan/playground/hk_horse_racing/project_tools/project_utils.py:540: RuntimeWarning: divide by zero encountered in scalar divide
  elif operator == 'division':


generating statistic horse_id_race_course_track_running_position_3_sum
generating statistic horse_id_race_course_track_running_position_3_mean
generating statistic horse_id_race_course_track_count
generating statistic horse_id_race_course_track_running_position_3_std
generating statistic horse_id_race_course_track_running_position_3_consistency


/home/yifan/playground/hk_horse_racing/project_tools/project_utils.py:540: RuntimeWarning: divide by zero encountered in scalar divide
  elif operator == 'division':


generating statistic horse_id_race_course_track_running_position_4_sum
generating statistic horse_id_race_course_track_running_position_4_mean
generating statistic horse_id_race_course_track_count
generating statistic horse_id_race_course_track_running_position_4_std
generating statistic horse_id_race_course_track_running_position_4_consistency


/home/yifan/playground/hk_horse_racing/project_tools/project_utils.py:540: RuntimeWarning: divide by zero encountered in scalar divide
  elif operator == 'division':


In [319]:
res_dfs.keys()

dict_keys(['horse_id_race_course_track'])

In [320]:
horse_track_stat = res_dfs['horse_id_race_course_track']

In [321]:
horse_track_stat.sample(5)

,horse_id,race_course_track,horse_id_race_course_track_running_position_1_sum,horse_id_race_course_track_running_position_1_mean,horse_id_race_course_track_count,horse_id_race_course_track_running_position_1_std,horse_id_race_course_track_running_position_1_consistency,horse_id_race_course_track_running_position_2_sum,horse_id_race_course_track_running_position_2_mean,horse_id_race_course_track_running_position_2_std,horse_id_race_course_track_running_position_2_consistency,horse_id_race_course_track_running_position_3_sum,horse_id_race_course_track_running_position_3_mean,horse_id_race_course_track_running_position_3_std,horse_id_race_course_track_running_position_3_consistency,horse_id_race_course_track_running_position_4_sum,horse_id_race_course_track_running_position_4_mean,horse_id_race_course_track_running_position_4_std,horse_id_race_course_track_running_position_4_consistency
1868,N100,"Happy Valley_TURF - ""B"" COURSE",2.0,2.0,0,NaN,NaN,2.0,2.0,NaN,NaN,4.0,4.0,NaN,NaN,0.0,NaN,NaN,NaN
710,A201,"Sha Tin_TURF - ""A"" COURSE",7.0,7.0,1,NaN,NaN,8.0,8.0,NaN,NaN,7.0,7.0,NaN,NaN,8.0,8.0,NaN,NaN
11700,V188,"Sha Tin_TURF - ""B+2"" COURSE",7.0,7.0,1,NaN,NaN,7.0,7.0,NaN,NaN,7.0,7.0,NaN,NaN,7.0,7.0,NaN,NaN
1510,M325,Sha Tin_ALL WEATHER TRACK,29.0,5.8,2,3.033150,1.912203,30.0,6.0,3.082207,1.946657,32.0,6.4,3.781534,1.692435,11.0,5.5,4.949747,1.111168
11891,V224,"Sha Tin_TURF - ""C"" COURSE",21.0,10.5,1,3.535534,2.969848,20.0,10.0,5.656854,1.767767,22.0,11.0,2.828427,3.889087,6.0,6.0,NaN,NaN


In [239]:
horse_feats.sample(5)

,horse_id,length_behind_winner_numeric
8977,P178,0.50
19919,V231,1.00
29553,V407,2.25
14029,S397,10.75
17567,P308,9.25


In [264]:
horse_df['clean_actual_weight'].value_counts()

clean_actual_weight
126.0    2057
133.0    1951
123.0    1739
125.0    1738
120.0    1605
122.0    1490
121.0    1469
124.0    1437
127.0    1373
128.0    1356
119.0    1294
131.0    1289
118.0    1266
129.0    1220
130.0    1214
117.0    1176
115.0    1173
116.0    1158
132.0    1004
114.0     894
113.0     849
112.0     400
111.0     371
108.0     168
109.0     146
110.0     143
107.0      78
106.0      48
105.0      39
104.0      24
103.0      18
Name: count, dtype: int64

In [257]:
horse_df['actual_weight'].value_counts()

actual_weight
126    2057
133    1951
123    1739
125    1738
120    1605
122    1490
121    1469
124    1437
127    1373
128    1356
119    1294
131    1289
118    1266
129    1220
130    1214
117    1176
115    1173
116    1158
132    1004
114     894
113     849
112     400
111     371
108     168
109     146
110     143
107      78
106      48
105      39
104      24
103      18
-         2
Name: count, dtype: int64

In [258]:
horse_df['declared_horse_weight'].value_counts()

declared_horse_weight
1088    229
1094    222
1106    217
1089    217
1108    214
       ... 
919       1
905       1
902       1
917       1
1325      1
Name: count, Length: 399, dtype: int64

In [259]:
horse_df['declared_horse_weight'].values[0]

'1032'

# initial analysis

In [289]:
print(horse_df.shape)
print(race_df.shape)


(30189, 25)
(2367, 13)


In [290]:
print(horse_df.columns)
print(race_df.columns)


Index(['finishing_position', 'horse_number', 'horse_name', 'horse_id',
       'jockey', 'trainer', 'actual_weight', 'declared_horse_weight', 'draw',
       'length_behind_winner', 'running_position_1', 'running_position_2',
       'running_position_3', 'running_position_4', 'finish_time', 'win_odds',
       'running_position_5', 'running_position_6', 'race_id', 'clean_position',
       'clean_actual_weight', 'clean_declared_horse_weight',
       'clean_length_behind_winner', 'isin_winner', 'isin_top3'],
      dtype='object')
Index(['src', 'race_date', 'race_course', 'race_number', 'race_id',
       'race_class', 'race_distance', 'track_condition', 'race_name', 'track',
       'sectional_time', 'incident_report', 'race_course_track'],
      dtype='object')


In [8]:
horse_df.sample(5)

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,running_position_1,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id
28604,13,7.0,SOUTHERN LEGEND,A252,J Moreira,C Fownes,122,1121,1,7-1/2,6.0,5.0,13.0,NaN,1.10.68,2.7,NaN,NaN,2016-688
19859,13,3.0,MAJOR CONNECTION,V067,O Bosson,D J Hall,132,1079,5,9-3/4,8.0,14.0,13.0,NaN,1.11.08,40,NaN,NaN,2015-774
22911,12,1.0,CELESTIAL NOVA,V045,T H So,C S Shum,131,1142,5,12-3/4,7.0,8.0,12.0,NaN,1.11.86,99,NaN,NaN,2016-230
11364,13,10.0,GOLDEN OSMANTHUS,N245,C K Tong,T K Ng,120,1138,9,11,7.0,8.0,10.0,13.0,1.24.99,99,NaN,NaN,2015-112
26937,8,5.0,JE PENSE,V354,S Clipperton,C W Chang,126,1111,7,6,6.0,6.0,6.0,6.0,1.49.47,11,8.0,NaN,2016-546


In [ ]:
horse_df['running_position_1']

In [ ]:
use_cols=['']
horse_df.sample(5)

In [33]:
horse_df['length_behind_winner'].sample(5)

25304    2-1/4
15497        -
24597    6-1/2
1829     6-1/2
4088     3-1/2
Name: length_behind_winner, dtype: object

In [39]:
horse_df['length_behind_winner'].unique().tolist()

['-',
 '2',
 '4-1/4',
 '5-1/2',
 '5-3/4',
 '6-1/4',
 '6-3/4',
 '7-3/4',
 '10-1/4',
 '11',
 '---',
 '2-1/4',
 '2-1/2',
 '2-3/4',
 '3-1/2',
 '4',
 '4-1/2',
 '4-3/4',
 '5',
 '7-1/4',
 '9',
 '10-3/4',
 '1-1/4',
 '1-3/4',
 '7',
 '8-1/2',
 '12-1/2',
 'N',
 '1',
 '3-3/4',
 '6',
 '3',
 '3-1/4',
 '5-1/4',
 '16-1/4',
 '20',
 'SH',
 'HD',
 '1/2',
 '7-1/2',
 '8-1/4',
 '12-1/4',
 '17-1/2',
 '1-1/2',
 '11-1/4',
 '12-3/4',
 '8',
 '6-1/2',
 '9-1/4',
 '9-1/2',
 '10-1/2',
 '21-1/4',
 '36-1/4',
 '13-3/4',
 '10',
 '3/4',
 '11-1/2',
 '24',
 '25-1/4',
 '12',
 '16',
 '26',
 '8-3/4',
 '16-1/2',
 '11-3/4',
 '13-1/4',
 '14-3/4',
 '15',
 '29-1/4',
 '19-1/2',
 '9-3/4',
 '19-1/4',
 '22-1/4',
 '22-1/2',
 '13-1/2',
 '16-3/4',
 '17',
 '14',
 '13',
 '22',
 '15-1/4',
 '39',
 '20-3/4',
 '22-3/4',
 '19-3/4',
 '21-3/4',
 '55-1/2',
 '14-1/2',
 '14-1/4',
 '15-1/2',
 '26-3/4',
 '20-1/4',
 '30',
 '38-1/2',
 'NOSE',
 '46-1/4',
 '18-3/4',
 '33-3/4',
 '24-3/4',
 '15-3/4',
 '23',
 '19',
 '21-1/2',
 '23-3/4',
 '29-3/4',
 '27-3/4',

In [208]:
lbw_vals = horse_df['length_behind_winner'].unique().tolist()
non_numeric_lbw = ['---', '-', 'N', 'SH', 'HD', 'NOSE', '+NOSE', 'ML', '+SH', 'TO', '+1/2']
numeric_lbw = [v for v in lbw_vals if v not in non_numeric_lbw]
print(numeric_lbw)

['2', '4-1/4', '5-1/2', '5-3/4', '6-1/4', '6-3/4', '7-3/4', '10-1/4', '11', '2-1/4', '2-1/2', '2-3/4', '3-1/2', '4', '4-1/2', '4-3/4', '5', '7-1/4', '9', '10-3/4', '1-1/4', '1-3/4', '7', '8-1/2', '12-1/2', '1', '3-3/4', '6', '3', '3-1/4', '5-1/4', '16-1/4', '20', '1/2', '7-1/2', '8-1/4', '12-1/4', '17-1/2', '1-1/2', '11-1/4', '12-3/4', '8', '6-1/2', '9-1/4', '9-1/2', '10-1/2', '21-1/4', '36-1/4', '13-3/4', '10', '3/4', '11-1/2', '24', '25-1/4', '12', '16', '26', '8-3/4', '16-1/2', '11-3/4', '13-1/4', '14-3/4', '15', '29-1/4', '19-1/2', '9-3/4', '19-1/4', '22-1/4', '22-1/2', '13-1/2', '16-3/4', '17', '14', '13', '22', '15-1/4', '39', '20-3/4', '22-3/4', '19-3/4', '21-3/4', '55-1/2', '14-1/2', '14-1/4', '15-1/2', '26-3/4', '20-1/4', '30', '38-1/2', '46-1/4', '18-3/4', '33-3/4', '24-3/4', '15-3/4', '23', '19', '21-1/2', '23-3/4', '29-3/4', '27-3/4', '21', '28-1/4', '25', '18-1/2', '32', '32-1/2', '27-1/2', '42', '26-1/4', '31-1/2', '40-1/2', '33-1/2', '17-1/4', '17-3/4', '18', '28-3/4', '

In [209]:
lbw_t2n_dict = {}
lbw_t2n_dict['-'] = 0
lbw_t2n_dict['N'] = 0.3
lbw_t2n_dict['SH'] = 0.1
lbw_t2n_dict['HD'] = 0.25
lbw_t2n_dict['NOSE'] = 0.05
lbw_t2n_dict['+NOSE'] = 0.75
lbw_t2n_dict['ML'] = 50
lbw_t2n_dict['+SH'] = 0.15
lbw_t2n_dict['TO'] = 0.1
lbw_t2n_dict['+1/2'] = 99
lbw_t2n_dict['---'] = 99

lbw_t2n_dict

{'-': 0,
 'N': 0.3,
 'SH': 0.1,
 'HD': 0.25,
 'NOSE': 0.05,
 '+NOSE': 0.75,
 'ML': 50,
 '+SH': 0.15,
 'TO': 0.1,
 '+1/2': 99,
 '---': 99}

In [207]:
# to check ---, -, 'N', 'SH', 'HD', 'NOSE', '+NOSE', 'ML', '+SH', 'TO', '+1/2'
# --- 99

lbw_t2n_dict = {}
lbw_t2n_dict['-'] = 0
lbw_t2n_dict['N'] = 0.3
lbw_t2n_dict['SH'] = 0.1
lbw_t2n_dict['HD'] = 0.25
lbw_t2n_dict['NOSE'] = 0.05
lbw_t2n_dict['+NOSE'] = 0.75
lbw_t2n_dict['ML'] = 50
lbw_t2n_dict['+SH'] = 0.15
lbw_t2n_dict['TO'] = 0.1
lbw_t2n_dict['+1/2'] = 99
lbw_t2n_dict['---'] = 99


    

use_cols = ['finishing_position', 'clean_position', 'length_behind_winner']
non_numeric_lbw = ['---', '-', 'N', 'SH', 'HD', 'NOSE', '+NOSE', 'ML', '+SH', 'TO', '+1/2']
# horse_df[horse_df['length_behind_winner'].isin(non_numeric_lbw)][use_cols].sample(5)
horse_df[horse_df['length_behind_winner']=='+1/2'][use_cols]

,finishing_position,clean_position,length_behind_winner
4456,DISQ,99,+1/2


In [42]:


use_cols = ['finishing_position', 'clean_position', 'length_behind_winner']
horse_df[horse_df['length_behind_winner']=='NOSE'][use_cols].sample(5)

,clean_position,length_behind_winner
22300,2,NOSE
24924,2,NOSE
18179,2,NOSE
9241,2,NOSE
25543,2,NOSE


In [100]:
horse_df[horse_df['clean_position']==4][use_cols].sample(5)

,clean_position,length_behind_winner
7906,4,2-3/4
18671,4,2
14241,4,2-1/4
14837,4,4-1/4
4806,4,1-3/4


In [36]:
horse_df['clean_position'].describe()

count    30189.000000
mean         8.862334
std         14.070619
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         99.000000
Name: clean_position, dtype: float64

In [87]:
pd.isnull(horse_df['finishing_position']).sum()

0

In [88]:
horse_df['finishing_position'].value_counts()

finishing_position
1        2361
2        2354
3        2350
6        2346
5        2341
4        2340
7        2339
8        2329
9        2305
10       2260
11       2186
12       2020
13        995
14        838
WV        461
WV-A      102
4 DH       35
PU         29
UR         26
3 DH       20
5 DH       18
WX         16
2 DH       14
6 DH       14
8 DH       12
1 DH       12
FE         10
WX-A       10
10 DH       8
9 DH        8
7 DH        8
DNF         7
TNP         6
11 DH       4
12 DH       2
NA          2
DISQ        1
Name: count, dtype: int64

In [89]:
horse_df[horse_df['finishing_position'].str.contains('DH')]['finishing_position']

297      9 DH
298      9 DH
592      2 DH
593      2 DH
752      1 DH
         ... 
29328    9 DH
29665    5 DH
29666    5 DH
29945    4 DH
29946    4 DH
Name: finishing_position, Length: 155, dtype: object

In [79]:
horse_df['running_position_3'].describe()

count    29542.000000
mean         6.826992
std          3.724886
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         14.000000
Name: running_position_3, dtype: float64

In [80]:
horse_df['running_position_4'].describe()

count    16618.000000
mean         6.942472
std          3.798080
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         14.000000
Name: running_position_4, dtype: float64

In [77]:
horse_df['jockey'].value_counts()

jockey
J Moreira    1995
K Teetan     1761
Z Purton     1651
D Whyte      1608
N Callan     1590
             ... 
K Manning       1
T Ono           1
M Nunes         1
J Spencer       1
T Jarnet        1
Name: count, Length: 106, dtype: int64

In [78]:
horse_df['trainer'].value_counts()

trainer
A S Cruz          1751
C Fownes          1724
C H Yip           1692
J Moore           1654
Y S Tsui          1652
J Size            1499
C S Shum          1466
W Y So            1438
P F Yiu           1410
A T Millard       1349
K L Man           1337
T P Yung          1263
D E Ferraris      1251
P O'Sullivan      1195
A Lee             1182
C W Chang         1136
K W Lui           1132
R Gibson          1103
D J Hall          1090
D Cruz            1012
L Ho              1012
T K Ng             715
S Woods            509
A Schutz           496
H Fujiwara           8
N Hori               7
Y Ikee               5
D A Hayes            4
A Fabre              4
A P O'Brien          4
J Lau                3
M Saito              3
Sir M R Stoute       3
Barande-Barbe        3
P Bary               2
G Allendorf          2
K C Chong            2
Y C Fung             2
de Royer Dupre       2
G Enebish            2
M C Tam              2
M Sakaguchi          2
T Ozeki              2
K C

In [37]:
race_df['incident_report'].sample(1).values[0]

'\n                TRAVEL BROTHER was withdrawn on 5.4.16 by order of the Stewards acting on veterinary advice (lame left fore).  BORN IN CHINA was also withdrawn on 5.4.16 by order of the Stewards acting on veterinary advice (lame right fore).  Before being allowed to race again, TRAVEL BROTHER and BORN IN CHINA will be subjected to an official veterinary examination.\nFrom the outside barriers, MR GENUINE and DISCIPLES TWELVE were shifted across behind runners shortly after the start.\nApproaching the 800 Metres, MR GENUINE was retired from the race.  A veterinary inspection of MR GENUINE immediately following the race found that horse to be lame in its right hind leg.  Before being allowed to race again, MR GENUINE will be subjected to an official veterinary examination.\nPABLOSKY and PACKING LLAREGYB were sent for sampling.\n'

In [278]:
exclude_cols =['incident_report']
race_df.drop(exclude_cols, axis=1).sample(5)

,src,race_date,race_course,race_number,race_id,race_class,race_distance,track_condition,race_name,track,sectional_time,race_course_track,GOOD TO FIRM_ratio,WET FAST_ratio,GOOD_ratio,GOOD TO YIELDING_ratio,WET SLOW_ratio,FAST_ratio,YIELDING_ratio,YIELDING TO SOFT_ratio,SOFT_ratio
2256,20170604-1.html,2017-06-04,Sha Tin,1,2016-697,Griffin Race,1200,GOOD TO FIRM,TATE'S CAIRN PLATE,"TURF - ""B"" COURSE",25.54 23.15 22.28,"Sha Tin_TURF - ""B"" COURSE",0.142857,0.00000,0.857143,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2151,20170423-3.html,2017-04-23,Sha Tin,3,2016-591,Class 4,1650,WET SLOW,AGE-FRIENDLY CITY HANDICAP,ALL WEATHER TRACK,27.96 23.43 24.20 23.44,Sha Tin_ALL WEATHER TRACK,0.000000,0.04878,0.808362,0.000000,0.076655,0.066202,0.000000,0.0,0.0
731,20150627-6.html,2015-06-27,Sha Tin,6,2014-731,Class 3,1000,GOOD TO FIRM,LEI YUE MUN PUBLIC RIDING SCHOOL HANDICAP,"TURF - ""C+3"" COURSE",13.05 20.42 22.02,"Sha Tin_TURF - ""C+3"" COURSE",0.533865,0.00000,0.406375,0.051793,0.000000,0.000000,0.007968,0.0,0.0
1267,20160320-10.html,2016-03-20,Sha Tin,10,2015-499,Class 3,1600,GOOD,COLLECTION HANDICAP,"TURF - ""A"" COURSE",25.10 23.09 24.18 23.09,"Sha Tin_TURF - ""A"" COURSE",0.550781,0.00000,0.382812,0.050781,0.000000,0.000000,0.015625,0.0,0.0
1228,20160302-8.html,2016-03-02,Happy Valley,8,2015-452,Class 3,1650,GOOD,SIU SAI WAN HANDICAP,"TURF - ""C+3"" COURSE",28.00 23.31 25.07 23.81,"Happy Valley_TURF - ""C+3"" COURSE",0.342105,0.00000,0.638158,0.019737,0.000000,0.000000,0.000000,0.0,0.0


In [51]:
race_df['track_condition'].value_counts()

track_condition
GOOD                1296
GOOD TO FIRM         860
GOOD TO YIELDING     126
YIELDING              28
WET SLOW              22
FAST                  19
WET FAST              14
YIELDING TO SOFT       1
SOFT                   1
Name: count, dtype: int64

In [53]:
race_df['race_course'].value_counts()


race_course
Sha Tin         1513
Happy Valley     854
Name: count, dtype: int64

In [74]:
race_df['race_class'].value_counts()

race_class
Class 4                        859
Class 3                        756
Class 5                        339
Class 2                        223
Class 1                         51
Group One                       32
Hong Kong Group Three           25
Griffin Race                    16
Group Two                       14
Group Three                     12
Class 4 (Restricted)             9
Hong Kong Group One              9
Hong Kong Group Two              8
Restricted Race                  6
Class 4 (Special Condition)      6
Class 3 (Special Condition)      2
Name: count, dtype: int64

In [72]:
race_df['race_name'].sample(5)

408                                          KUT CHEONG HANDICAP
2362                                         MIRACULOUS HANDICAP
1967                                          BUTTERFLY HANDICAP
642     THE SPORTS CLUB DIAMOND JUBILEE CHALLENGE CUP (HANDICAP)
1177                                            HEATHER HANDICAP
Name: race_name, dtype: object

In [54]:
race_df['race_name'].value_counts()

race_name
LYNDHURST HANDICAP                                     6
SATURN HANDICAP                                        6
KAM TIN RIVER HANDICAP                                 5
WONG NAI CHUNG HANDICAP                                5
DANDELION HANDICAP                                     5
                                                      ..
TONG FUK HANDICAP                                      1
AUDEMARS PIGUET HANDICAP                               1
THE PRINCE JEWELLERY & WATCH PREMIER CUP (HANDICAP)    1
VACHERON CONSTANTIN HANDICAP                           1
MEDIC KINGDOM HANDICAP                                 1
Name: count, Length: 1084, dtype: int64

In [76]:
race_df['race_course_track'].value_counts()

race_course_track
Sha Tin_ALL WEATHER TRACK           287
Happy Valley_TURF - "A" COURSE      264
Sha Tin_TURF - "A" COURSE           256
Sha Tin_TURF - "C+3" COURSE         251
Sha Tin_TURF - "C" COURSE           242
Happy Valley_TURF - "B" COURSE      236
Sha Tin_TURF - "B+2" COURSE         207
Happy Valley_TURF - "C" COURSE      202
Sha Tin_TURF - "A+3" COURSE         200
Happy Valley_TURF - "C+3" COURSE    152
Sha Tin_TURF - "B" COURSE            70
Name: count, dtype: int64

In [50]:
# sentiment analysis on hourse 

rid = np.random.randint(1, len(race_df))
print(race_df['incident_report'][rid])


                AUDACITY was crowded for room on jumping between TEAM SWEET and FOREVER ACCURATE which shifted in.
Shortly after the start, GAINFULJET and REGENCY BABY bumped, resulting in both horses becoming badly unbalanced.
From the outside barrier, SUPER SWEET ORANGE was shifted across behind runners in the early stages.
After the 1200 Metres, SUPER SWEET ORANGE got its head up on a number of occasions when proving difficult to settle.
After the 800 Metres, SMART SALUTE was left racing wide and without cover.
Near the 500 Metres, WORLD RECORD got its head on the side and shifted out towards the heels of SUPER SWEET ORANGE.
Shortly after entering the Straight, DIVINE DIYA was steadied when disappointed for running between TEAM SWEET and EQUITY DOCTRINE.
The Stewards inquired into the reason for Apprentice K K Chiong being dislodged from SUPER MAN passing the 350 Metres.  After taking evidence from Apprentice Chiong, in the presence of her allocated trainer, Mr K W Lui, N Callan (G